##專案介紹摘要
本專案以 Hugging Face 上的 mT5 模型為基礎，建構一個多語言文案生成器，使用者可輸入產品描述、自選語言與文案風格，由模型生成對應語言的行銷文案。介面使用 Gradio 架設，提升互動性與實用性。

##實作重點
使用google/mt5-small，支援超過 100 種語言
prompt設計格式：「為下列產品撰寫【語言】的【風格】文案」
開放使用者自行選擇語言與風格，提供互動式體驗
可擴充到 mBART 或其他多語言模型




1.安裝套件

In [10]:
!pip install transformers sentencepiece gradio

# 載入 mBART 模型與 tokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import gradio as gr

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# 行銷文案生成函數
def generate_copy(description, language, style):
    # 對應語言代碼
    lang_code_map = {
        "英文": "en_XX",
        "中文": "zh_CN",
        "日文": "ja_XX",
        "法文": "fr_XX",
        "德文": "de_DE"
    }
    tgt_lang = lang_code_map.get(language, "en_XX")
    model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

    # prompt 可簡化為符合語言模型理解的格式
    prompt = f"{style}行銷文案：{description}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
        temperature=1.0
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

2.Gradio

In [11]:
# Gradio UI
demo = gr.Interface(
    fn=generate_copy,
    inputs=[
        gr.Textbox(label="產品描述", placeholder="例如：保濕精華，適合乾燥肌膚"),
        gr.Dropdown(choices=["英文", "日文", "法文", "德文", "中文"], label="目標語言"),
        gr.Radio(choices=["熱情推銷型", "簡約風格", "故事敘述型"], label="文案風格")
    ],
    outputs="text",
    title="🌍 多語言文案生成器",
    description="輸入產品描述、選擇語言與文案風格，自動生成行銷文案！",
    theme="default"
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc5459bf6af6a5b8d6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
